In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from geonamescache import GeonamesCache
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap

In [2]:
gc = GeonamesCache()
iso3_codes = list(gc.get_dataset_by_key(gc.get_countries(), 'iso3').keys())

filename = 'extdebt.csv'
shapefile = 'shp/countries/ne_110m_admin_0_countries_lakes'

In [3]:
df = pd.read_csv(filename, skiprows=4)
df.set_index('Country Code', inplace=True)

In [6]:
num_colors = 9
values = df['2010']
cm = plt.get_cmap('Blues')
scheme = cm(1.*np.arange(num_colors)/num_colors)
bins = np.linspace(values.min(), values.max(), num_colors)
df['bin'] = np.digitize(values, bins) - 1
mpl.style.use('seaborn-white')
fig = plt.figure(figsize=(11, 6))

ax = fig.add_subplot(111, axisbg='w', frame_on=False)
fig.suptitle('Test', fontsize=30, y=.95)

m = Basemap(lon_0=0, projection='robin')
m.drawmapboundary(color='#ffffff')

m.readshapefile(shapefile, 'units', linewidth=0)
for info, shape in zip(m.units_info, m.units):
    iso3 = info['adm0_a3']
    if iso3 not in df.index:
        color = '#dddddd'
    else:
        color = scheme[df.ix[iso3]['bin']]

    patches = [Polygon(np.array(shape), True)]
    pc = PatchCollection(patches, edgecolor='#7c7c7c', lw=.2, alpha=1.)
    pc.set_facecolor(color)
    ax.add_collection(pc)

plt.savefig('test.png', bbox_inches='tight', pad_inches=.2, dpi=200)